# Assistants RAG Demo

This is a basic demo that uses the OpenAI assistants API to answer questions about
Coda's help desk. It's very minimal at this point, so feel free to tweak it to your needs!

<a target="_blank" href="https://colab.research.google.com/github/braintrustdata/braintrust-examples/blob/main/help-docs/py/Assistants_Help_Desk.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
%env OPENAI_API_KEY=<YOUR_OPENAI_KEY>
%env BRAINTRUST_API_KEY=<YOUR_BRAINTRUST_KEY>

In [ ]:
%pip install braintrust markdownify openai pydantic requests

# Contants

Feel free to tweak these constants to scale up & down

In [ ]:
QA_GEN_MODEL="gpt-3.5-turbo"
ASSISTANTS_MODEL="gpt-4-1106-preview"
NUM_SECTIONS = 20
NUM_DOCS = 20
NUM_QUESTIONS = 20

# Download data

First, let's download the data and split it into markdown sections

In [ ]:
import markdownify
import re
import requests

data = requests.get(
    "https://gist.githubusercontent.com/wong-codaio/b8ea0e087f800971ca5ec9eef617273e/raw/39f8bd2ebdecee485021e20f2c1d40fd649a4c77/articles.json"
).json()
markdown_docs = [{"id": row["id"], "markdown": markdownify.markdownify(row["body"])} for row in data]

i = 0
markdown_sections = []
for markdown_doc in markdown_docs:
    sections = re.split(r"(.*\n=+\n)", markdown_doc["markdown"])
    current_section = ""
    for section in sections:
        if not section.strip():
            continue

        if re.match(r".*\n=+\n", section):
            current_section = section
        else:
            section = current_section + section
            markdown_sections.append({"doc_id": markdown_doc["id"], "section_id": i, "markdown": section.strip()})
            current_section = ""
            i += 1

# Generate QA pairs

We'll iterate through several sections and generate reference QA pairs to test.

In [ ]:
from openai import AsyncOpenAI
import os

openai = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
import asyncio
import json
from pydantic import BaseModel, Field
from typing import List


class QAPair(BaseModel):
    question: str = Field(
        ..., description="Question"
    )
    answer: str = Field(..., description="Answer")


class QAPairs(BaseModel):
    pairs: List[QAPair] = Field(..., description="List of question/answer pairs")


async def produce_candidate_questions(row):
    response = await openai.chat.completions.create(
        model=QA_GEN_MODEL,
        messages=[{"role": "assistant", "content": f"""
Please generate 8 question/answer pairs from the following text.

Content:

{row['markdown']}
"""}],
        functions=[
            {
                "name": "propose_qa_pairs",
                "description": "Propose some question/answer pairs for a given document",
                "parameters": QAPairs.schema(),
            }
        ],
    )

    pairs = QAPairs(**json.loads(response.choices[0].message.function_call.arguments))
    return pairs.pairs


all_candidates_futures = [
    asyncio.create_task(produce_candidate_questions(a)) for a in markdown_sections[:NUM_SECTIONS]
]
all_candidates = [await f for f in all_candidates_futures]

In [ ]:
all_candidates[0]

# Initialize the assistant, and load the files in

In [ ]:
import tempfile

tempdir = tempfile.TemporaryDirectory()

markdown_files = []
for i, d in enumerate(markdown_docs[:NUM_DOCS]):
  fname = os.path.join(tempdir.name, f"{i}.md")
  with open(fname, "w") as f:
    f.write(d["markdown"])
  markdown_files.append(await openai.files.create(file=open(fname, "rb"), purpose="assistants"))
  print(i)



In [ ]:
len(markdown_docs)

In [ ]:
assistant = await openai.beta.assistants.create(
    name="Help Desk Bot",
    instructions="You are a support assistant. Answer questions from the Help Desk using the provided documents",
    tools=[{"type": "retrieval"}],
    model=ASSISTANTS_MODEL,
    file_ids=[f.id for f in markdown_files],
)


Let's ask a basic question

In [ ]:
all_candidates[0][0].question

In [ ]:
thread = await openai.beta.threads.create()
message = await openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=all_candidates[0][0].question,
)

In [ ]:
run = await openai.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)
run

In [ ]:
import time

start = time.time()
while run.completed_at is None and run.failed_at is None:
  if time.time() - start > 60:
    print(run)
    raise Exception("Run did not finish after 1 minute")
  run = await openai.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

print("Took ", time.time()-start, " seconds to receive message")

In [ ]:
json.dumps(run.dict())

In [ ]:
if run.failed_at:
  print("FAIL", run)
else:
  messages = await openai.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)

# Running an eval

Now let's package this up and run an evaluation.

In [ ]:
from braintrust import current_span

async def answer_question(input):
  thread = await openai.beta.threads.create()
  message = await openai.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=input,
  )
  run = await openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
  )
  start = time.time()
  while run.completed_at is None and run.failed_at is None:
    if time.time() - start > 60:
      print(run)
      raise Exception("Run did not finish after 1 minute")
    run = await openai.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )

  current_span().log(metadata={
      "run": run.dict()
  })
  if run.failed_at:
    return None
  else:
    messages = await openai.beta.threads.messages.list(
      thread_id=thread.id
    )
    return messages.data[0].content[0].text.value

print(await answer_question("How do I create a formula?"))

In [ ]:
from autoevals import Factuality
from braintrust import Eval

def load_data():
  return [
      {
          "input": qa_pair.question,
          "expected": qa_pair.answer,
      }
      for section in all_candidates
      for qa_pair in section
  ][:NUM_QUESTIONS]

await Eval(
    "assistants-help-desk",
    data=load_data,
    task=answer_question,
    scores=[Factuality]
)